<a href="https://colab.research.google.com/github/jjkcoding/Airplane-Prices/blob/main/Airplane_Prices_Analysis_and_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Questions: \\
1) Which day is the cheapest day to depart? \\
2) Does increase in difference in days increase the price? \\
3) How much does distance change in price?

Problems: \\
1) Web browser would take a while to load, so webscraping took a couple hours. \\
2) Prices change constantly \\
3) Could not find api or website with previous airplane pricing data. Most likely due to airplane prices fluctutate depending on the day of purchase.

Todo: \\
1) Name graphs \\
2) Write the purpose for each graph \\
3) Logistic Regression or ANOVA with tukey \\
4) maybe assumptions? \\
5) create logistic regression  model? \\
6) Select 100 random points for testing from training data

In [ ]:
# install chromium, its driver, and selenium:
!apt-get update -y &> /dev/null
!apt install chromium-chromedriver &> /dev/null
!cp /usr/lib/chromium-browser/chromedriver/usr/bin &> /dev/null
!pip install selenium &> /dev/null

import numpy as np
import pandas as pd
import requests
import plotly.express as px
from lxml import html
from time import sleep
from selenium import webdriver
from datetime import datetime
from datetime import timedelta

user_agent = 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.50 Safari/537.36'

options = webdriver.ChromeOptions()
options.add_argument('-headless')
options.add_argument('-no-sandbox')
options.add_argument('-disable-dev-shm-usage')

options.add_argument(f'user-agent={user_agent}')

In [ ]:
# Departure Airport | Arriving Airport | Depart day | Arrival day | Day Diff | Day Type (M, T, W, ..., Su) | Amount ($) | Distance (LATER)
def webscrape_data(beg_str, end_str, airport_from, airport_to):
  kayak_web = ('https://www.kayak.com/flights/' + airport_from + '-' + airport_to +
             '/' + beg_str + '-flexible/' + end_str + '?sort=bestflight_a')
  kayak_content = requests.get(kayak_web)
  kayak_tree = html.fromstring(kayak_content.content)
  wd.get(kayak_web)
  sleep(50)
  data_text = wd.find_element_by_xpath("//div[@style='flex:7']/div[@role='rowgroup']").text
  return data_text.split('\n')

def update_df(depart_date, arrive_date, airport_from, airport_to, money_to_int, df):
  for i in range(0, len(money_to_int)):
    cur_day = depart_date + timedelta(days = -3 + i)
    new_row = {'Depart_Airport':airport_from, 'Arrive_Airport':airport_to, 
               'Depart_Date':cur_day, 'Arrival_Date':arrive_date, 
               'Depart_Day_Type':cur_day.strftime('%A'), 
               'Arrive_Day_Type':arrive_date.strftime('%A'), 
               'Cost':money_to_int[i]}
    df = df.append(new_row, ignore_index = True)
  return df


# Gather info from 3 days before depart date to 1 day before arrival date with a constant arrival date
def gather_info_everyday(depart_date, arrive_date, airport_from, airport_to, df):
  while depart_date < arrive_date:
    money_vals = webscrape_data(depart_date.strftime("%Y-%m-%d"), arrive_date.strftime("%Y-%m-%d"), airport_from, airport_to)[:min((arrive_date - depart_date).days + 3, 7)]
    money_to_int = list(map(lambda x: int(x[1:].replace(",","")), money_vals))
    df = update_df(depart_date, arrive_date, airport_from, airport_to, money_to_int, df)
    depart_date += timedelta(days = 7)
  return df 


# Gather info from every four days
def gather_info_every_four(depart_date, arrive_date, airport_from, airport_to, df):
  cur_arrive_date = depart_date + timedelta(days = 4)
  while depart_date < arrive_date:
    money_vals = webscrape_data(depart_date.strftime("%Y-%m-%d"), 
                                cur_arrive_date.strftime("%Y-%m-%d"), 
                                airport_from, 
                                airport_to)[:min((arrive_date - depart_date).days + 3, 7)]
    money_to_int = list(map(lambda x: int(x[1:].replace(",","")), money_vals))
    df = update_df(depart_date, cur_arrive_date, airport_from, airport_to, money_to_int, df)
    df.to_excel("final_update.xlsx")
    depart_date += timedelta(days = 4)
    cur_arrive_date += timedelta(days=4)
  return df 



start: 2021-07-01 to end: 2022-04-01 \\
total days: 274

In [ ]:
wd = webdriver.Chrome('chromedriver',options=options)
sleep(2)

depart_str = '2021-07-01'
arrive_str = '2022-04-01'
airport_from = 'LAX'
airport_to = 'SEL'
depart_date = pd.to_datetime(depart_str)
arrive_date = pd.to_datetime(arrive_str)

air_dat = pd.DataFrame()
air_dat = gather_info_every_four(depart_date, arrive_date, airport_from, airport_to, air_dat)

In [ ]:
air_dat = pd.read_excel("/content/final_update.xlsx", index_col = 0)
air_dat = air_dat[['Depart_Airport', 'Arrive_Airport', 'Depart_Date', 'Arrival_Date', 'Depart_Day_Type', 'Arrive_Day_Type', 'Cost']]
air_dat

,Depart_Airport,Arrive_Airport,Depart_Date,Arrival_Date,Depart_Day_Type,Arrive_Day_Type,Cost
0,LAX,SEL,2021-06-28,2021-07-05,Monday,Monday,1415
1,LAX,SEL,2021-06-29,2021-07-05,Tuesday,Monday,1420
2,LAX,SEL,2021-06-30,2021-07-05,Wednesday,Monday,1112
3,LAX,SEL,2021-07-01,2021-07-05,Thursday,Monday,1404
4,LAX,SEL,2021-07-02,2021-07-05,Friday,Monday,1465
...,...,...,...,...,...,...,...
475,LAX,SEL,2022-03-30,2022-03-31,Wednesday,Thursday,1058
476,LAX,SEL,2022-03-28,2022-04-04,Monday,Monday,805
477,LAX,SEL,2022-03-29,2022-04-04,Tuesday,Monday,805
478,LAX,SEL,2022-03-30,2022-04-04,Wednesday,Monday,805


In [ ]:
air_dat = air_dat.assign(Day_Diff = lambda x: (x.Arrival_Date - x.Depart_Date).dt.days)
air_dat['Depart_Month'] = list(map(lambda x: x.month, air_dat['Depart_Date']))
air_dat['Arrival_Month'] = list(map(lambda x: x.month, air_dat['Arrival_Date']))
air_dat

,Depart_Airport,Arrive_Airport,Depart_Date,Arrival_Date,Depart_Day_Type,Arrive_Day_Type,Cost,Day_Diff,Depart_Month,Arrival_Month
0,LAX,SEL,2021-06-28,2021-07-05,Monday,Monday,1415,7,6,7
1,LAX,SEL,2021-06-29,2021-07-05,Tuesday,Monday,1420,6,6,7
2,LAX,SEL,2021-06-30,2021-07-05,Wednesday,Monday,1112,5,6,7
3,LAX,SEL,2021-07-01,2021-07-05,Thursday,Monday,1404,4,7,7
4,LAX,SEL,2021-07-02,2021-07-05,Friday,Monday,1465,3,7,7
...,...,...,...,...,...,...,...,...,...,...
475,LAX,SEL,2022-03-30,2022-03-31,Wednesday,Thursday,1058,1,3,3
476,LAX,SEL,2022-03-28,2022-04-04,Monday,Monday,805,7,3,4
477,LAX,SEL,2022-03-29,2022-04-04,Tuesday,Monday,805,6,3,4
478,LAX,SEL,2022-03-30,2022-04-04,Wednesday,Monday,805,5,3,4


In [ ]:
depart_plt = px.box(air_dat, x = 'Depart_Day_Type', y = 'Cost',
                    category_orders={"Depart_Day_Type": ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]})
depart_plt.show()

arrive_plt = px.box(air_dat, x = 'Arrive_Day_Type', y = 'Cost',
                    category_orders={"Arrive_Day_Type": ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]})
arrive_plt.show()

day_diff_plt = px.box(air_dat, x = 'Day_Diff', y = 'Cost',
                      category_orders={"Day_Diff": [1, 2, 3, 4, 5, 6, 7]})
day_diff_plt.show()


In [ ]:
day_diff_by_depart_plt = px.box(air_dat, x = 'Day_Diff', y = 'Cost', color = 'Depart_Day_Type',
                                category_orders={"Depart_Day_Type": ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]})
day_diff_by_depart_plt.show()


day_diff_by_arrive_plt = px.box(air_dat, x = 'Day_Diff', y = 'Cost', color = 'Arrive_Day_Type',
                                category_orders={"Arrive_Day_Type": ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]})
day_diff_by_arrive_plt.show()

In [ ]:
depart_month_plt = px.box(air_dat, x = 'Depart_Month', y = 'Cost',
                          category_orders={"Depart_Month": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]})
depart_month_plt.show()

arrival_month_plt = px.box(air_dat, x = 'Arrival_Month', y = 'Cost',
                           category_orders={"Arrival_Month": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]})
arrival_month_plt.show()

In [ ]:
depart_month_day_diff_plt = px.box(air_dat, x = 'Depart_Month', y = 'Cost', color = 'Day_Diff',
                          category_orders={"Depart_Month": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12],
                                           "Day_Diff":[1, 2, 3, 4, 5, 6, 7]})
depart_month_day_diff_plt.show()

depart_month_and_day_type_plt = px.box(air_dat, x = 'Depart_Month', y = 'Cost', color = 'Depart_Day_Type',
                          category_orders={"Depart_Month": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12],
                                           "Depart_Day_Type": ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]})
depart_month_and_day_type_plt.show()

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import LabelEncoder
le1 = LabelEncoder()
le2 = LabelEncoder()
model = LinearRegression()
air_dat

,Depart_Airport,Arrive_Airport,Depart_Date,Arrival_Date,Depart_Day_Type,Arrive_Day_Type,Cost,Day_Diff,Depart_Month,Arrival_Month
0,LAX,SEL,2021-06-28,2021-07-05,Monday,Monday,1415,7,6,7
1,LAX,SEL,2021-06-29,2021-07-05,Tuesday,Monday,1420,6,6,7
2,LAX,SEL,2021-06-30,2021-07-05,Wednesday,Monday,1112,5,6,7
3,LAX,SEL,2021-07-01,2021-07-05,Thursday,Monday,1404,4,7,7
4,LAX,SEL,2021-07-02,2021-07-05,Friday,Monday,1465,3,7,7
...,...,...,...,...,...,...,...,...,...,...
475,LAX,SEL,2022-03-30,2022-03-31,Wednesday,Thursday,1058,1,3,3
476,LAX,SEL,2022-03-28,2022-04-04,Monday,Monday,805,7,3,4
477,LAX,SEL,2022-03-29,2022-04-04,Tuesday,Monday,805,6,3,4
478,LAX,SEL,2022-03-30,2022-04-04,Wednesday,Monday,805,5,3,4


In [ ]:
dfle = air_dat.copy()
dfle.Depart_Day_Type = le1.fit_transform(dfle.Depart_Day_Type)
dfle.Arrive_Day_Type = le2.fit_transform(dfle.Arrive_Day_Type)
dfle


,Depart_Airport,Arrive_Airport,Depart_Date,Arrival_Date,Depart_Day_Type,Arrive_Day_Type,Cost,Day_Diff,Depart_Month,Arrival_Month
0,LAX,SEL,2021-06-28,2021-07-05,1,1,1415,7,6,7
1,LAX,SEL,2021-06-29,2021-07-05,5,1,1420,6,6,7
2,LAX,SEL,2021-06-30,2021-07-05,6,1,1112,5,6,7
3,LAX,SEL,2021-07-01,2021-07-05,4,1,1404,4,7,7
4,LAX,SEL,2021-07-02,2021-07-05,0,1,1465,3,7,7
...,...,...,...,...,...,...,...,...,...,...
475,LAX,SEL,2022-03-30,2022-03-31,6,4,1058,1,3,3
476,LAX,SEL,2022-03-28,2022-04-04,1,1,805,7,3,4
477,LAX,SEL,2022-03-29,2022-04-04,5,1,805,6,3,4
478,LAX,SEL,2022-03-30,2022-04-04,6,1,805,5,3,4


In [ ]:
X = dfle[['Depart_Day_Type', 'Arrive_Day_Type', 'Depart_Month', 'Arrival_Month', 'Day_Diff']].values
X

In [ ]:
y = dfle.Cost
y

In [ ]:
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder()

In [ ]:
X_tran = ohe.fit_transform(X).toarray()
X_tran

In [ ]:
X_tran = X_tran[:, 1:]
X_tran

In [ ]:
model.fit(X_tran, y)

In [ ]:
model.predict([X_tran[479, :]])

In [ ]:
model.score(X_tran, y)